<a href="https://colab.research.google.com/github/JonathanRibeiro92/ReconhecimentoPadroes/blob/main/tarefa_ensembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataset: steel-plates-fault
[Dataset OpenML - 1504](https://www.openml.org/search?type=data&sort=runs&status=active&qualities.NumberOfInstances=between_1000_10000&qualities.NumberOfClasses=%3D_2&id=1504)

Um conjunto de dados de falhas em chapas de aço, classificadas em 7 tipos diferentes.

In [1]:
from sklearn.datasets import fetch_openml

dataset = fetch_openml(data_id=1504)
dataset.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [7]:
import pandas

X, y = dataset.data, pandas.to_numeric(dataset.target)
X.shape, y.shape

((1941, 33), (1941,))

In [9]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=42)
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((1455, 33), (486, 33), (1455,), (486,))

##Desabilitar warnings

In [5]:
import warnings

warnings.filterwarnings('ignore')

##Combinação de Classificadores

###Voting

In [13]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

modelo = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('log', LogisticRegression()),
    ('gnb', GaussianNB())
])

modelo.fit(Xtrain, ytrain)
vote_pred = modelo.predict(Xtest)
vote_hits = vote_pred == ytest
resultadoVoting = sum(vote_hits)/len(vote_hits)
print(f'Resultado Voting:{resultadoVoting}')

Resultado Voting:0.6481481481481481


###Stacking

In [25]:
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

stacking = StackingClassifier([
    ('knn', KNeighborsClassifier()),
    ('log', LogisticRegression()),
    ('gnb', GaussianNB())
])

stacking.fit(Xtrain, ytrain)
stacking_pred = stacking.predict(Xtest)
stacking_hits = stacking_pred == ytest
resultadoStacking = sum(stacking_hits)/len(stacking_hits)
print(f'Resultado Stacking:{resultadoStacking}')

Resultado Stacking:0.6502057613168725


###Stacking + Voting

In [26]:
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

voting = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('log', LogisticRegression()),
    ('gnb', GaussianNB())
])

stackingVoting = StackingClassifier([
    ('voting', voting),
    ('extrat', ExtraTreesClassifier()),
    ('ranfor', RandomForestClassifier())
], cv=3, passthrough=True)

stackingVoting.fit(Xtrain, ytrain)
stacking_pred = stackingVoting.predict(Xtest)
stacking_pred_hits = stacking_pred == ytest
resultadoStackingVoting = sum(stacking_pred_hits)/len(stacking_pred_hits)
print(f'Resultado Stacking+Voting:{resultadoStackingVoting}')

Resultado Stacking+Voting:0.6481481481481481


##Usando GridSearch

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import numpy as np


grid_params = {
    'voting': ['soft', 'hard'],
}

grid_search = GridSearchCV(voting, param_grid=grid_params, n_jobs= 8)
grid_search.fit(X, y)
cross_val = np.mean(cross_val_score(grid_search, X, y))
print(cross_val, grid_search.best_params_, grid_search.best_score_)

0.5775726817374712 {'voting': 'hard'} 0.5775726817374712
